In [10]:
!pip install --upgrade datasets transformers evaluate
!pip install --upgrade evaluate


#from transformers import BertTokenizer, BertForQuestionAnswering, AdamW
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, TrainingArguments, Trainer
import numpy as np
import evaluate

# Load SQuAD 1.1 dataset
squad = load_dataset("squad")

# Initialize tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = AutoModelForQuestionAnswering.from_pretrained("distilbert-base-uncased")

# Extract questions and contexts from the dataset
questions = squad['train']['question']
contexts = squad['train']['context']

# Tokenize questions and contexts
inputs = tokenizer(questions, contexts, truncation=True, padding=True, max_length=384)



tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
def compute_metrics(eval_pred):
    """Compute Exact Match (EM) and F1 scores."""
    predictions, labels = eval_pred
    metric = load_metric("squad")
    formatted_predictions = [
        {"id": str(i), "prediction_text": pred} for i, pred in enumerate(predictions)
    ]
    formatted_references = [
        {"id": str(i), "answers": {"text": [label], "answer_start": [0]}} for i, label in enumerate(labels)
    ]
    results = metric.compute(predictions=formatted_predictions, references=formatted_references)
    return results

In [20]:
!pip uninstall transformers -y
!pip install transformers
!pip install --upgrade accelerate

from transformers import TrainingArguments

def train_qa_system():
    """Train and evaluate a QA system."""
    from datasets import load_dataset
    from transformers import AutoTokenizer, AutoModelForQuestionAnswering, TrainingArguments, Trainer

    # Load SQuAD 1.1 dataset
    squad = load_dataset("squad")

    # Choose a pretrained model
    model_name = "distilbert-base-uncased"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForQuestionAnswering.from_pretrained(model_name)

    # Preprocess the dataset
    def preprocess_data(examples):
        tokenized_examples = tokenizer(
            examples["question"],
            examples["context"],
            truncation=True,
            padding="max_length",
            max_length=384,
            return_offsets_mapping=True,
        )
        
        # Start and end positions
        start_positions = []
        end_positions = []
        
        for i, answer in enumerate(examples["answers"]):
            if len(answer["answer_start"]) > 0:  
                start_pos = answer["answer_start"][0]
                end_pos = start_pos + len(answer["text"][0]) - 1
            else:  # If no answer, use special tokens
                start_pos = 0
                end_pos = 0
            start_positions.append(start_pos)
            end_positions.append(end_pos)
        
        tokenized_examples["start_positions"] = start_positions
        tokenized_examples["end_positions"] = end_positions
        return tokenized_examples

    tokenized_squad = squad.map(preprocess_data, batched=True, remove_columns=squad["train"].column_names)

    # Define training arguments
    training_args = TrainingArguments(
        output_dir="./results",
        evaluation_strategy="epoch",
        learning_rate=2e-5,
        per_device_train_batch_size=16,
        num_train_epochs=3,
        weight_decay=0.01,
        save_strategy="epoch",
        load_best_model_at_end=True,
        metric_for_best_model="exact_match",
        report_to="none",
    )

    # Define the Trainer
    def compute_metrics(eval_pred):
        from datasets import load_metric
        metric = load_metric("squad")
        predictions, labels = eval_pred
        return metric.compute(predictions=predictions, references=labels)

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_squad["train"],
        eval_dataset=tokenized_squad["validation"],
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,
    )

    # Train and Evaluate
    trainer.train()
    evaluation_results = trainer.evaluate()
    print("Evaluation Results:")
    print(evaluation_results)

if __name__ == "__main__":
    train_qa_system()


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Found existing installation: transformers 4.47.0
Uninstalling transformers-4.47.0:
  Successfully uninstalled transformers-4.47.0


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached transformers-4.47.0-py3-none-any.whl.metadata (43 kB)
Using cached transformers-4.47.0-py3-none-any.whl (10.1 MB)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.26.0`: Please run `pip install transformers[torch]` or `pip install 'accelerate>={ACCELERATE_MIN_VERSION}'`